# Otto RecSys - Candidate ReRank Model - Co-visit generator

## **Tổng quan Giải pháp: Mô hình Candidate-ReRank với Luật Tự chế**

Notebook này triển khai một mô hình gợi ý theo kiến trúc 2 giai đoạn kinh điển: **Tạo Ứng viên (Candidate Generation)** và **Tái xếp hạng (Re-ranking)**. Điểm đặc biệt của giải pháp này là giai đoạn Re-ranking được thực hiện hoàn toàn bằng các **Luật Tự chế (Handcrafted Rules)** thay vì dùng một mô hình Machine Learning.

---

### **Giai đoạn 1: Tạo Ứng viên (Candidate Generation)**

Mục tiêu của giai đoạn này là tạo ra một danh sách các sản phẩm tiềm năng (ứng viên) cho mỗi người dùng từ nhiều nguồn khác nhau. Để tăng tốc độ xử lý trên bộ dữ liệu lớn, notebook sử dụng thư viện **RAPIDS cuDF** để tính toán trên GPU.

Các nguồn ứng viên chính bao gồm:

1.  **Lịch sử Tương tác của Người dùng (User History):**
    *   Gợi ý lại chính những sản phẩm mà người dùng đã từng click, thêm vào giỏ, hoặc mua. Đây là nguồn tín hiệu cá nhân hóa mạnh nhất.

2.  **Các Sản phẩm Phổ biến nhất (Popular Items):**
    *   Top 20 sản phẩm được `click` hoặc `order` nhiều nhất trong toàn bộ tập dữ liệu. Đây là nguồn gợi ý dự phòng, đặc biệt hữu ích cho các người dùng mới.

3.  **Ma trận Co-visitation 1 ("Carts/Orders" - Có trọng số loại):**
    *   **Mục tiêu:** Dự đoán các hành động có ý định mua hàng cao (`carts`, `orders`).
    *   **Logic:** Trả lời câu hỏi: "Nếu người dùng đã tương tác với sản phẩm A, họ có khả năng sẽ *thêm vào giỏ/mua* sản phẩm B nào nhất?". Các hành động `cart` và `order` được gán trọng số cao hơn để nhấn mạnh tầm quan trọng.

4.  **Ma trận Co-visitation 2 ("Buy2Buy"):**
    *   **Mục tiêu:** Tìm các mối liên hệ mua hàng chất lượng cao nhất.
    *   **Logic:** Trả lời câu hỏi: "Nếu người dùng đã *mua* (hoặc thêm vào giỏ) sản phẩm A, họ có khả năng sẽ *mua* (hoặc thêm vào giỏ) sản phẩm B nào nhất?". Ma trận này hoàn toàn bỏ qua các hành động `click` để giảm nhiễu.

5.  **Ma trận Co-visitation 3 ("Clicks" - Có trọng số thời gian):**
    *   **Mục tiêu:** Dự đoán các hành động khám phá (`clicks`) và nắm bắt xu hướng.
    *   **Logic:** Trả lời câu hỏi: "Nếu người dùng đã tương tác với sản phẩm A, họ có khả năng sẽ *click* sản phẩm B nào nhất?". Các tương tác xảy ra gần đây được gán trọng số cao hơn để ưu tiên các xu hướng mới.

---

### **Giai đoạn 2: Tái xếp hạng & Lựa chọn (Re-ranking)**

Sau khi có danh sách ứng viên từ các nguồn trên, giai đoạn này sẽ sắp xếp lại chúng và chọn ra 20 gợi ý cuối cùng cho mỗi loại (`clicks`, `carts`, `orders`). Giai đoạn này được hiện thực hóa bằng hai hàm riêng biệt (`suggest_clicks` và `suggest_buys`) với các quy tắc ưu tiên thủ công:

1.  **Tính gần đây (Recency):** Ưu tiên các sản phẩm trong lịch sử được người dùng tương tác gần nhất.
2.  **Sự lặp lại (Repetition):** Các sản phẩm được người dùng tương tác nhiều lần sẽ được đẩy lên cao hơn.
3.  **Ý định Mua hàng (Intent):** Các sản phẩm đã từng được `cart` hoặc `order` có độ ưu tiên cao.
4.  **Tín hiệu Chất lượng cao:** Các gợi ý đến từ ma trận "Buy2Buy" được ưu tiên trong việc dự đoán `carts`/`orders`.
5.  **Xử lý Đặc biệt:** Đối với những người dùng có lịch sử phong phú (tương tác với >= 20 sản phẩm), hệ thống sẽ chủ yếu dựa vào việc phân tích lịch sử của chính họ (kết hợp recency, repetition, và intent) thay vì dùng nhiều các gợi ý từ co-visitation.

Đây là một baseline mạnh mẽ với phần tạo ứng viên rất chất lượng. Cơ hội cải tiến lớn nhất nằm ở việc thay thế các **Luật Tự chế** ở Giai đoạn 2 bằng một mô hình Machine Learning (ví dụ: LGBMRanker) để có thể "học" được các quy luật xếp hạng phức tạp hơn từ dữ liệu.

## Scripts
Tạo ra các scripts/source code/functions tái sử dụng được, tuân thủ DRY.

In [1]:
from collections import Counter
import gc
import glob
import itertools
import os
import pickle
import sys

import cudf
import numpy as np
import pandas as pd
import polars as pl
from tqdm.notebook import tqdm

In [2]:
print(cudf.__version__)

25.02.02


In [3]:
TYPE_LABELS_MAPPING = {
    'clicks': 0,
    'carts': 1,
    'orders': 2,
}

### Cache file

In [4]:
_DATA_CACHE = dict()

def read_file(file_path: str):
    global _DATA_CACHE
    return cudf.DataFrame(_DATA_CACHE[file_path])

def load_file(file_path: str):
    global _DATA_CACHE
    df = pd.read_parquet(file_path)
    # Reduce timestamp memory
    df['ts'] = (df['ts'] / 1000).astype('int32')
    df['type'] = df['type'].map(TYPE_LABELS_MAPPING).astype('int8')
    _DATA_CACHE[file_path] = df

In [5]:
def cache_files(files: list[str]):
    for file in files:
        load_file(file)

In [6]:
def clear_cache():
    global _DATA_CACHE
    _DATA_CACHE.clear()

### Constants/Config

In [7]:
# Constants
READ_CT = 5
NUM_CHUNK = 6
FIX_SIZE = 1.86e6

CO_VISIT_MATRIX_OUT_PATH_TEMPLATE = '{output_dir}/top_{top_k}_{matrix}_{part}.pqt'

### Pre-calculate co-visit matrix

#### Cart-Order Co-visit Matrix: Type Weighted
Tạo ra một ma trận co-visitation trả lời câu hỏi: "Nếu một người dùng đã tương tác (click/cart/order) với sản phẩm aid_x, thì họ có khả năng sẽ thêm vào giỏ (cart) hoặc mua (order) sản phẩm aid_y nào nhất?".

In [8]:
def calculate_carts_orders_matrix(
    files: list[str],
    disk_pieces: int,
    last_hold_aids_each_session: int,
    valid_time: int,
    top_k: int,
    type_weight: dict,
    save_parquet: bool=False,
    output_dir: str|None=None
) -> dict:
    piece_size = FIX_SIZE / disk_pieces
    chunk_size = int(np.ceil(len(files) / NUM_CHUNK))
    saved_files = []
    for part in range(disk_pieces):
        # Mỗi vòng to sẽ xử lý 1/4 dữ liệu
        print('='*20)
        print(f'### DISK PART {part + 1}')
    
        for chunk_idx in range(NUM_CHUNK):
            # Qua từng chunk nhỏ để giảm RAM
            start_file_idx = chunk_idx * chunk_size
            end_file_idx = min((chunk_idx + 1) * chunk_size, len(files))
            print(f'Processing from file {start_file_idx} to {end_file_idx} in group of {READ_CT}')
    
            for k in range(start_file_idx, end_file_idx, READ_CT):
                # Inner chunk
                # Đọc 5 file một lúc từ cache, đẩy lên GPU và nối lại
                df = [read_file(files[k])]
                for i in range(1, READ_CT):
                    if k + i < end_file_idx: 
                        df.append(read_file(files[k+i]))
                df = cudf.concat(df, ignore_index=True, axis=0)
    
                # Sắp xếp theo session và thời gian (giảm dần)
                df = df.sort_values(['session','ts'], ascending=[True,False])
                
                # --- Tối ưu hóa quan trọng ---
                # Chỉ giữ lại 30 hành động cuối cùng của mỗi session
                df = df.reset_index(drop=True)
                df['n'] = df.groupby('session').cumcount()
                df = df.loc[df.n<last_hold_aids_each_session].drop('n',axis=1)
    
                # Tự merge một dataframe với chính nó dựa trên session
                # Kết quả: tạo ra tất cả các cặp (aid_x, aid_y) có thể có trong cùng 1 session
                df = df.merge(df, on='session')
    
                # --- Lọc các cặp không hợp lệ ---
                # 1. Chỉ giữ các cặp có thời gian cách nhau không quá 1 ngày (24*60*60 giây)
                # 2. Loại bỏ các cặp mà aid_x giống hệt aid_y
                df = df.loc[((df.ts_x - df.ts_y).abs()< valid_time) & (df.aid_x != df.aid_y)]
    
                # Chỉ tính cho các aid_x thuộc phần hiện tại
                df = df.loc[(df.aid_x >= part*piece_size) & (df.aid_x < (part+1)*piece_size)]
    
                # --- Gán trọng số và tính toán ---
                # Chỉ giữ các cột cần thiết và loại bỏ các cặp trùng lặp
                df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
                # Dùng .map() để gán trọng số dựa trên 'type' của sản phẩm thứ hai (aid_y)
                df['wgt'] = df.type_y.map(type_weight)
                df = df[['aid_x','aid_y','wgt']]
                df.wgt = df.wgt.astype('float32')
    
                # Nhóm theo cặp (aid_x, aid_y) và cộng tất cả các trọng số lại
                df = df.groupby(['aid_x','aid_y']).wgt.sum()
    
                # Combine inner chunk
                if k == start_file_idx:
                    inner_tmp = df
                else:
                    inner_tmp = inner_tmp.add(df, fill_value=0)
    
                print(k,', ',end='')
            print("="*10)
            # Combine outer chunk
            if start_file_idx == 0:
                outer_tmp = inner_tmp
            else:
                outer_tmp = outer_tmp.add(inner_tmp, fill_value=0)
            del inner_tmp, df
            gc.collect()
    
        # Convert matrix to dict
        outer_tmp = outer_tmp.reset_index()
        outer_tmp = outer_tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
    
        outer_tmp = outer_tmp.reset_index(drop=True)
        outer_tmp['n'] = outer_tmp.groupby('aid_x')['aid_y'].cumcount()
        outer_tmp = outer_tmp.loc[outer_tmp.n < top_k].drop('n', axis=1)
    
        # Save to disk
        if save_parquet:
            out_path = CO_VISIT_MATRIX_OUT_PATH_TEMPLATE.format(
                output_dir=output_dir if output_dir else '/kaggle/working',
                top_k=top_k,
                matrix='carts_orders',
                part=part
            )
            outer_tmp.to_pandas().to_parquet(out_path, index=None)
            saved_files.append(out_path)

    return {
        'num_part': len(saved_files),
        'matrix_type': 'carts_orders',
        'saved_files': saved_files,
        'top_k': top_k
    }

#### BUY2BUY Co-visit Matrix
Tạo ra một ma trận co-visitation trả lời câu hỏi: "Nếu một người dùng đã thêm vào giỏ (cart) hoặc mua (order) sản phẩm aid_x, thì họ có khả năng sẽ thêm vào giỏ hoặc mua sản phẩm aid_y nào nhất?".
Đây là tín hiệu về ý định mua hàng nghiêm túc, vì nó hoàn toàn bỏ qua các hành động "click" vốn có thể rất nhiễu (người dùng có thể click vào nhiều thứ nhưng không có ý định mua).

In [9]:
def calculate_buy2buy_matrix(
    files: list[str],
    disk_pieces: int,
    last_hold_aids_each_session: int,
    valid_time: int,
    top_k: int,
    save_parquet: bool=False,
    output_dir: str|None=None
):
    piece_size = FIX_SIZE / disk_pieces
    chunk_size = int(np.ceil(len(files) / NUM_CHUNK))
    saved_files = []
    for part in range(disk_pieces):
        # Mỗi vòng to sẽ xử lý 1/4 dữ liệu
        print('='*20)
        print(f'### DISK PART {part + 1}')
    
        for chunk_idx in range(NUM_CHUNK):
            # Qua từng chunk nhỏ để giảm RAM
            start_file_idx = chunk_idx * chunk_size
            end_file_idx = min((chunk_idx + 1) * chunk_size, len(files))
            print(f'Processing from file {start_file_idx} to {end_file_idx} in group of {READ_CT}')
    
            for k in range(start_file_idx, end_file_idx, READ_CT):
                # Inner chunk
                # Đọc 5 file một lúc từ cache, đẩy lên GPU và nối lại
                df = [read_file(files[k])]
                for i in range(1, READ_CT):
                    if k + i < end_file_idx: 
                        df.append(read_file(files[k+i]))
                df = cudf.concat(df, ignore_index=True, axis=0)
                df = df.loc[df['type'].isin([1,2])] # ONLY WANT CARTS AND ORDERS
                # Sắp xếp theo session và thời gian (giảm dần)
                df = df.sort_values(['session','ts'], ascending=[True,False])
                
                # --- Tối ưu hóa quan trọng ---
                # Chỉ giữ lại 30 hành động cuối cùng của mỗi session
                df = df.reset_index(drop=True)
                df['n'] = df.groupby('session').cumcount()
                df = df.loc[df.n<last_hold_aids_each_session].drop('n',axis=1)
    
                # Tự merge một dataframe với chính nó dựa trên session
                # Kết quả: tạo ra tất cả các cặp (aid_x, aid_y) có thể có trong cùng 1 session
                df = df.merge(df, on='session')
    
                # --- Lọc các cặp không hợp lệ ---
                # 1. Chỉ giữ các cặp có thời gian cách nhau không quá 1 ngày (24*60*60 giây)
                # 2. Loại bỏ các cặp mà aid_x giống hệt aid_y
                df = df.loc[((df.ts_x - df.ts_y).abs()< valid_time) & (df.aid_x != df.aid_y)]
    
                # Chỉ tính cho các aid_x thuộc phần hiện tại
                df = df.loc[(df.aid_x >= part*piece_size) & (df.aid_x < (part+1)*piece_size)]
    
                # --- Gán trọng số và tính toán ---
                # Chỉ giữ các cột cần thiết và loại bỏ các cặp trùng lặp
                df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
                # Dùng .map() để gán trọng số dựa trên 'type' của sản phẩm thứ hai (aid_y)
                df['wgt'] = 1
                df = df[['aid_x','aid_y','wgt']]
                df.wgt = df.wgt.astype('float32')
    
                # Nhóm theo cặp (aid_x, aid_y) và cộng tất cả các trọng số lại
                df = df.groupby(['aid_x','aid_y']).wgt.sum()
    
                # Combine inner chunk
                if k == start_file_idx:
                    inner_tmp = df
                else:
                    inner_tmp = inner_tmp.add(df, fill_value=0)
    
                print(k,', ',end='')
            print("="*10)
            # Combine outer chunk
            if start_file_idx == 0:
                outer_tmp = inner_tmp
            else:
                outer_tmp = outer_tmp.add(inner_tmp, fill_value=0)
            del inner_tmp, df
            gc.collect()
    
        # Convert matrix to dict
        outer_tmp = outer_tmp.reset_index()
        outer_tmp = outer_tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
    
        outer_tmp = outer_tmp.reset_index(drop=True)
        outer_tmp['n'] = outer_tmp.groupby('aid_x')['aid_y'].cumcount()
        outer_tmp = outer_tmp.loc[outer_tmp.n < top_k].drop('n', axis=1)
    
        # Save to disk
        if save_parquet:
            out_path = CO_VISIT_MATRIX_OUT_PATH_TEMPLATE.format(
                output_dir=output_dir if output_dir else '/kaggle/working',
                top_k=top_k,
                matrix='buy2buy',
                part=part
            )
            outer_tmp.to_pandas().to_parquet(out_path, index=None)
            saved_files.append(out_path)

    return {
        'num_part': len(saved_files),
        'matrix_type': 'buy2buy',
        'saved_files': saved_files,
        'top_k': top_k
    }

#### "Click" Co-visit Matrix - Time Weighted
Tạo ra một ma trận co-visitation trả lời câu hỏi: "Nếu một người dùng đã tương tác (click/cart/order) với sản phẩm aid_x, thì họ có khả năng sẽ click vào sản phẩm aid_y nào nhất, với điều kiện là các tương tác gần đây phải quan trọng hơn?".
Ma trận này được gọi là "Time Weighted" vì trọng số của mỗi cặp tương tác phụ thuộc vào thời điểm nó xảy ra.

In [10]:
def calculate_clicks_matrix(
    files: list[str],
    disk_pieces: int,
    last_hold_aids_each_session: int,
    valid_time: int,
    top_k: int,
    time_weighted_func: callable,
    save_parquet: bool=False,
    output_dir: str|None=None
):
    piece_size = FIX_SIZE / disk_pieces
    chunk_size = int(np.ceil(len(files) / NUM_CHUNK))
    saved_files = []
    for part in range(disk_pieces):
        # Mỗi vòng to sẽ xử lý 1/4 dữ liệu
        print('='*20)
        print(f'### DISK PART {part + 1}')
    
        for chunk_idx in range(NUM_CHUNK):
            # Qua từng chunk nhỏ để giảm RAM
            start_file_idx = chunk_idx * chunk_size
            end_file_idx = min((chunk_idx + 1) * chunk_size, len(files))
            print(f'Processing from file {start_file_idx} to {end_file_idx} in group of {READ_CT}')
    
            for k in range(start_file_idx, end_file_idx, READ_CT):
                # Inner chunk
                # Đọc 5 file một lúc từ cache, đẩy lên GPU và nối lại
                df = [read_file(files[k])]
                for i in range(1, READ_CT):
                    if k + i < end_file_idx: 
                        df.append(read_file(files[k+i]))
                df = cudf.concat(df, ignore_index=True, axis=0)
    
                # Sắp xếp theo session và thời gian (giảm dần)
                df = df.sort_values(['session','ts'], ascending=[True,False])
                
                # --- Tối ưu hóa quan trọng ---
                # Chỉ giữ lại 30 hành động cuối cùng của mỗi session
                df = df.reset_index(drop=True)
                df['n'] = df.groupby('session').cumcount()
                df = df.loc[df.n<last_hold_aids_each_session].drop('n',axis=1)
    
                # Tự merge một dataframe với chính nó dựa trên session
                # Kết quả: tạo ra tất cả các cặp (aid_x, aid_y) có thể có trong cùng 1 session
                df = df.merge(df, on='session')
    
                # --- Lọc các cặp không hợp lệ ---
                # 1. Chỉ giữ các cặp có thời gian cách nhau không quá 1 ngày (24*60*60 giây)
                # 2. Loại bỏ các cặp mà aid_x giống hệt aid_y
                df = df.loc[((df.ts_x - df.ts_y).abs()< valid_time) & (df.aid_x != df.aid_y)]
    
                # Chỉ tính cho các aid_x thuộc phần hiện tại
                df = df.loc[(df.aid_x >= part*piece_size) & (df.aid_x < (part+1)*piece_size)]
    
                # --- Gán trọng số và tính toán ---
                # Chỉ giữ các cột cần thiết và loại bỏ các cặp trùng lặp
                df = df[['session', 'aid_x', 'aid_y','type_y', 'ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y'])
                df['wgt'] = time_weighted_func(df['ts_x'])
                df = df[['aid_x','aid_y','wgt']]
                df.wgt = df.wgt.astype('float32')
    
                # Nhóm theo cặp (aid_x, aid_y) và cộng tất cả các trọng số lại
                df = df.groupby(['aid_x','aid_y']).wgt.sum()
    
                # Combine inner chunk
                if k == start_file_idx:
                    inner_tmp = df
                else:
                    inner_tmp = inner_tmp.add(df, fill_value=0)
    
                print(k,', ',end='')
            print("="*10)
            # Combine outer chunk
            if start_file_idx == 0:
                outer_tmp = inner_tmp
            else:
                outer_tmp = outer_tmp.add(inner_tmp, fill_value=0)
            del inner_tmp, df
            gc.collect()
    
        # Convert matrix to dict
        outer_tmp = outer_tmp.reset_index()
        outer_tmp = outer_tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
    
        outer_tmp = outer_tmp.reset_index(drop=True)
        outer_tmp['n'] = outer_tmp.groupby('aid_x')['aid_y'].cumcount()
        outer_tmp = outer_tmp.loc[outer_tmp.n < top_k].drop('n', axis=1)
    
        # Save to disk
        if save_parquet:
            out_path = CO_VISIT_MATRIX_OUT_PATH_TEMPLATE.format(
                output_dir=output_dir if output_dir else '/kaggle/working',
                top_k=top_k,
                matrix='clicks',
                part=part
            )
            outer_tmp.to_pandas().to_parquet(out_path)
            saved_files.append(out_path)

    return {
        'num_part': len(saved_files),
        'matrix_type': 'clicks',
        'saved_files': saved_files,
        'top_k': top_k
    }

### Full pipeline

In [11]:
import os

In [12]:
def full_pipeline(
    input_dir: str,
    output_dir: str,
    carts_orders_params: dict,
    buy2buy_params: dict,
    clicks_params: dict
):
    files = glob.glob(input_dir)
    cache_files(files)

    carts_orders_params.update({
        'files': files,
        'output_dir': output_dir
    })

    buy2buy_params.update({
        'files': files,
        'output_dir': output_dir
    })

    clicks_params.update({
        'files': files,
        'output_dir': output_dir
    })

    os.makedirs(output_dir, exist_ok=True)

    carts_orders_res = calculate_carts_orders_matrix(**carts_orders_params)
    buy2buy_res = calculate_buy2buy_matrix(**buy2buy_params)
    clicks_res = calculate_clicks_matrix(**clicks_params)

    print("Result: ")
    print(carts_orders_res)
    print(buy2buy_res)
    print(clicks_res)

    clear_cache()

## Prepare 3 co-visit matrix

In [13]:
carts_orders_params = {
    'disk_pieces': 4,
    'last_hold_aids_each_session': 30,
    'valid_time': 24 * 60 * 60, # 1 ngày
    'top_k': 15,
    'type_weight': {0: 1, 1: 6, 2: 3}, # 0:clicks, 1:carts, 2:orders
    'save_parquet': True,
}

buy2buy_params = {
    'disk_pieces': 1,
    'last_hold_aids_each_session': 30,
    'valid_time': 14 * 24 * 60 * 60, # 14 ngày
    'top_k': 15,
    'save_parquet': True,
}

from datetime import datetime
start_date = int(datetime(2022, 7, 31, 22, 0, 0).timestamp())
end_date = int(datetime(2022, 9, 4, 22, 0, 0).timestamp())

def linear_time_weighted(x):
    return 1 + 3 * (x - start_date) / (end_date - start_date)
    
clicks_params = {
    'disk_pieces': 4,
    'last_hold_aids_each_session': 30,
    'valid_time': 24 * 60 * 60, # 1 ngày
    'top_k': 20,
    'time_weighted_func': linear_time_weighted,
    'save_parquet': True,
}

### For submission pipeline

In [14]:
%%time
# Train
full_pipeline(
    '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/*',
    '/kaggle/working/submission_pipeline/train',
    carts_orders_params,
    buy2buy_params,
    clicks_params
)

### DISK PART 1
Processing from file 0 to 22 in group of 5
0 , 5 , 10 , 15 , 20 , ==========
Processing from file 22 to 44 in group of 5
22 , 27 , 32 , 37 , 42 , ==========
Processing from file 44 to 66 in group of 5
44 , 49 , 54 , 59 , 64 , ==========
Processing from file 66 to 88 in group of 5
66 , 71 , 76 , 81 , 86 , ==========
Processing from file 88 to 110 in group of 5
88 , 93 , 98 , 103 , 108 , ==========
Processing from file 110 to 129 in group of 5
110 , 115 , 120 , 125 , ==========
### DISK PART 2
Processing from file 0 to 22 in group of 5
0 , 5 , 10 , 15 , 20 , ==========
Processing from file 22 to 44 in group of 5
22 , 27 , 32 , 37 , 42 , ==========
Processing from file 44 to 66 in group of 5
44 , 49 , 54 , 59 , 64 , ==========
Processing from file 66 to 88 in group of 5
66 , 71 , 76 , 81 , 86 , ==========
Processing from file 88 to 110 in group of 5
88 , 93 , 98 , 103 , 108 , ==========
Processing from file 110 to 129 in group of 5
110 , 115 , 120 , 125 , ==========
### DI

In [15]:
%%time
# Full
full_pipeline(
    '/kaggle/input/otto-chunk-data-inparquet-format/*_parquet/*',
    '/kaggle/working/submission_pipeline/all',
    carts_orders_params,
    buy2buy_params,
    clicks_params
)

### DISK PART 1
Processing from file 0 to 25 in group of 5
0 , 5 , 10 , 15 , 20 , ==========
Processing from file 25 to 50 in group of 5
25 , 30 , 35 , 40 , 45 , ==========
Processing from file 50 to 75 in group of 5
50 , 55 , 60 , 65 , 70 , ==========
Processing from file 75 to 100 in group of 5
75 , 80 , 85 , 90 , 95 , ==========
Processing from file 100 to 125 in group of 5
100 , 105 , 110 , 115 , 120 , ==========
Processing from file 125 to 146 in group of 5
125 , 130 , 135 , 140 , 145 , ==========
### DISK PART 2
Processing from file 0 to 25 in group of 5
0 , 5 , 10 , 15 , 20 , ==========
Processing from file 25 to 50 in group of 5
25 , 30 , 35 , 40 , 45 , ==========
Processing from file 50 to 75 in group of 5
50 , 55 , 60 , 65 , 70 , ==========
Processing from file 75 to 100 in group of 5
75 , 80 , 85 , 90 , 95 , ==========
Processing from file 100 to 125 in group of 5
100 , 105 , 110 , 115 , 120 , ==========
Processing from file 125 to 146 in group of 5
125 , 130 , 135 , 140 , 1

### Local CV Pipeline

In [16]:
%%time
# Train
full_pipeline(
    '/kaggle/input/otto-validation/train_parquet/*',
    '/kaggle/working/local_cv/train',
    carts_orders_params,
    buy2buy_params,
    clicks_params
)

### DISK PART 1
Processing from file 0 to 17 in group of 5
0 , 5 , 10 , 15 , ==========
Processing from file 17 to 34 in group of 5
17 , 22 , 27 , 32 , ==========
Processing from file 34 to 51 in group of 5
34 , 39 , 44 , 49 , ==========
Processing from file 51 to 68 in group of 5
51 , 56 , 61 , 66 , ==========
Processing from file 68 to 85 in group of 5
68 , 73 , 78 , 83 , ==========
Processing from file 85 to 100 in group of 5
85 , 90 , 95 , ==========
### DISK PART 2
Processing from file 0 to 17 in group of 5
0 , 5 , 10 , 15 , ==========
Processing from file 17 to 34 in group of 5
17 , 22 , 27 , 32 , ==========
Processing from file 34 to 51 in group of 5
34 , 39 , 44 , 49 , ==========
Processing from file 51 to 68 in group of 5
51 , 56 , 61 , 66 , ==========
Processing from file 68 to 85 in group of 5
68 , 73 , 78 , 83 , ==========
Processing from file 85 to 100 in group of 5
85 , 90 , 95 , ==========
### DISK PART 3
Processing from file 0 to 17 in group of 5
0 , 5 , 10 , 15 , =====

In [17]:
%%time
# All
full_pipeline(
    '/kaggle/input/otto-validation/*_parquet/*',
    '/kaggle/working/local_cv/train',
    carts_orders_params,
    buy2buy_params,
    clicks_params
)

### DISK PART 1
Processing from file 0 to 20 in group of 5
0 , 5 , 10 , 15 , ==========
Processing from file 20 to 40 in group of 5
20 , 25 , 30 , 35 , ==========
Processing from file 40 to 60 in group of 5
40 , 45 , 50 , 55 , ==========
Processing from file 60 to 80 in group of 5
60 , 65 , 70 , 75 , ==========
Processing from file 80 to 100 in group of 5
80 , 85 , 90 , 95 , ==========
Processing from file 100 to 120 in group of 5
100 , 105 , 110 , 115 , ==========
### DISK PART 2
Processing from file 0 to 20 in group of 5
0 , 5 , 10 , 15 , ==========
Processing from file 20 to 40 in group of 5
20 , 25 , 30 , 35 , ==========
Processing from file 40 to 60 in group of 5
40 , 45 , 50 , 55 , ==========
Processing from file 60 to 80 in group of 5
60 , 65 , 70 , 75 , ==========
Processing from file 80 to 100 in group of 5
80 , 85 , 90 , 95 , ==========
Processing from file 100 to 120 in group of 5
100 , 105 , 110 , 115 , ==========
### DISK PART 3
Processing from file 0 to 20 in group of 5
0